In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from   scipy import stats
from   copy import deepcopy
import os
import sys
import glob

sys.path.append('/Users/research/projects/alderaan/')
from alderaan.utils import weighted_percentile

# Generate synthetic posteriors

In [ ]:
NPL = 100

mu_true = stats.beta(0.9, 3.0).rvs(NPL)

data = []

for i in range(NPL):
    mu = mu_true[i]
    
    sd_p = 0.03
    sd_n = 0.2
    
    d = pd.DataFrame(columns=['precise noisy try_sd try_mu'.split()])
    d.precise = stats.truncnorm(-mu/sd_p, (1-mu)/sd_p, loc=mu, scale=sd_p).rvs(3000)
    d.noisy   = stats.truncnorm(-mu/sd_n, (1-mu)/sd_n, loc=mu, scale=sd_n).rvs(3000)
    d.try_sd  = stats.norm(loc=mu, scale=sd_n).rvs(3000)
    d.try_mu  = stats.norm(loc=mu, scale=sd_n).rvs(3000) + np.random.normal(0, sd_n)
    
    data.append(d)

## Run the hierarchical model

In [ ]:
import aesara_theano_fallback.tensor as T
from   aesara_theano_fallback import aesara as theano
from   celerite2.theano import GaussianProcess
from   celerite2.theano import terms as GPterms
import pymc3 as pm
import pymc3_ext as pmx
import corner

#### Beta distribution

In [ ]:
def BetaDistPDF(a, b, x, B=None):
    '''
    The beta function B can be precomputed to improve performance
    This is necessary if looping over multiple realizations w/in a PyMC model
    '''
    if B is None:
        B = T.gamma(a)*T.gamma(b)/T.gamma(a+b)
        
    return x**(a-1) * (1-x)**(b-1) / B

In [ ]:
def BetaDistLogPDF(a, b, x, ln_B=None):
    '''
    The ln(beta) function ln_B can be precomputed to improve performance
    This is necessary if looping over multiple realizations w/in a PyMC model
    '''
    if ln_B is None:
        ln_B = T.gammaln(a) + T.gammaln(b) - T.gammaln(a+b)
        
    return (a-1)*np.log(x) + (b-1)*np.log(1-x) - ln_B

In [ ]:
NPL = len(data)
Nbin = 100

bin_edges = np.linspace(0, 1, Nbin+1)
bin_widths = bin_edges[1:] - bin_edges[:-1] 
bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])

with pm.Model() as model:
    # hyperpriors from Gelman Baysian Data Analysis Chapter 5
    x = pm.Uniform("x", lower=0.01, upper=0.99)                     # x = a/(a+b) = mean
    y = pm.Uniform("y", lower=0.1, upper=10)                        # y = 1/sqrt(a+b) ~ 'inverse precision'
    
    a = pm.Deterministic("a", x/y**2)
    b = pm.Deterministic("b", (1-x)/y**2)
    
    # precompute the beta function
    ln_B = T.gammaln(a) + T.gammaln(b) - T.gammaln(a+b)
    
    # track the pdf for convenience
    ln_pdf = pm.Deterministic("ln_pdf", BetaDistLogPDF(a, b, bin_centers, ln_B=ln_B))
    
    X = [None]*NPL
    C = [None]*NPL
    Z = T.zeros(NPL, dtype='float')
    
    for i, d in enumerate(data):
        X[i] = BetaDistLogPDF(a, b, d.noisy.values, ln_B=ln_B)
        C[i] = T.max(X[i])
        Z = T.set_subtensor(Z[i], C[i] + T.log(T.sum(T.exp(X[i]-C[i]))))
    
    # likelihood
    pm.Potential("ln_like", T.sum(Z))

In [ ]:
with model:
    trace = pmx.sample(tune=2000, draws=1000, chains=2, target_accept=0.9, return_inferencedata=True)

In [ ]:
x = np.exp(trace.posterior.ln_pdf.values)
x = x.reshape(-1,x.shape[-1])

plt.figure()
plt.fill_between(bin_centers, np.percentile(x, 16, axis=0), np.percentile(x, 84, axis=0), color='r', alpha=0.3)
plt.plot(bin_centers, np.median(x, axis=0), color='r', lw=2, label='posterior')
plt.plot(bin_centers, stats.beta(0.9,3.0).pdf(bin_centers), 'k--')
plt.ylim(0, None)
plt.xlim(0, 1)
plt.yticks([])
plt.xlabel("e", fontsize=24)
plt.ylabel("P(e)", fontsize=24)
plt.legend(fontsize=16)
plt.show()

In [ ]:
_ = corner.corner(trace, var_names=['x', 'y'], truths=[0.9/3.9, 1/np.sqrt(3.9)])

In [ ]:
_ = corner.corner(trace, var_names=['a', 'b'], truths=[0.9, 3.9])

#### Normal distribution

In [ ]:
def NormDistLogPDF(mu, sd, x):        
    return -T.log(sd) -0.5*np.log(2*np.pi) - (x-mu)**2/(2*sd**2)

In [ ]:
NPL = len(data)
Nbin = 100

bin_edges = np.linspace(0, 1, Nbin+1)
bin_widths = bin_edges[1:] - bin_edges[:-1] 
bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])

with pm.Model() as model:
    # hyperpriors
    mu = pm.Uniform("mu", lower=-1.0, upper=1.0)
    sd = pm.HalfCauchy("sd", beta=2.0)
            
    # track the pdf for convenience
    ln_pdf = pm.Deterministic("ln_pdf", NormDistLogPDF(mu, sd, bin_centers))
    
    X = [None]*NPL
    C = [None]*NPL
    Z = T.zeros(NPL, dtype='float')
    
    for i, d in enumerate(data):
        X[i] = NormDistLogPDF(mu, sd, d.ECC.values)
        C[i] = T.max(X[i])
        Z = T.set_subtensor(Z[i], C[i] + T.log(T.sum(T.exp(X[i]-C[i]))))
    
    # likelihood
    pm.Potential("ln_like", T.sum(Z))

In [ ]:
with model:
    trace = pmx.sample(tune=2000, draws=1000, chains=2, target_accept=0.9, return_inferencedata=True)

In [ ]:
x = np.exp(trace.posterior.ln_pdf.values)
x = x.reshape(-1,x.shape[-1])

plt.figure()
plt.hist(truth.ecc, color='lightgrey', bins=np.linspace(0,1,11), density=True, label='truth')
plt.fill_between(bin_centers, np.percentile(x, 16, axis=0), np.percentile(x, 84, axis=0), color='r', alpha=0.3)
plt.plot(bin_centers, np.median(x, axis=0), color='r', lw=2, label='posterior')
plt.plot(bin_centers, stats.beta(0.9,3.0).pdf(bin_centers), 'k--')
plt.ylim(0, None)
plt.xlim(0, 1)
plt.yticks([])
plt.xlabel("e", fontsize=24)
plt.ylabel("P(e)", fontsize=24)
plt.legend(fontsize=16)
plt.show()

In [ ]:
_ = corner.corner(trace, var_names=['mu', 'sd'])

#### Non-parametric

In [ ]:
NPL = len(data)
Nbin = 100

bin_edges = np.linspace(-2, 2, Nbin+1)
bin_widths = bin_edges[1:] - bin_edges[:-1] 
bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])

with pm.Model() as model:
    # hyperpriors on GP
    log_s = pm.Uniform("log_s", lower=-1, upper=5, testval=0)
    log_r = pm.Uniform("log_r", lower=-1, upper=5, testval=0)
    kernel = GPterms.Matern32Term(sigma=T.exp(log_s), rho=T.exp(log_r))
    
    # calculate bin heights from latent draws
    latent = pm.Normal("latent", mu=0, sd=1, shape=Nbin)
    LS = T.exp(log_s)*latent

    gp = GaussianProcess(kernel, mean=T.mean(LS))
    gp.compute(bin_centers, diag=T.var(LS[1:]-LS[:-1])/T.sqrt(2)*T.ones(Nbin))
    
    beta  = gp.predict(LS)
    ln_pdf = pm.Deterministic("ln_pdf", beta - T.log(T.sum(T.exp(beta)*bin_widths)))
    
    # hierarchical model
    X = [None]*NPL
    C = [None]*NPL
    Z = T.zeros(NPL, dtype='float')
    
    for i, d in enumerate(data):
        inds = np.digitize(d.try_mu.values, bin_edges[1:], right=True)
        X[i] = ln_pdf[inds]
        C[i] = T.max(X[i])
        Z = T.set_subtensor(Z[i], C[i] + T.log(T.sum(T.exp(X[i]-C[i]))))
    
    # likelihood
    pm.Potential("ln_like", T.sum(Z))

In [ ]:
with model:
    trace = pmx.sample(tune=2000, draws=1000, chains=2, target_accept=0.9, return_inferencedata=True)

In [ ]:
x = np.exp(trace.posterior.ln_pdf.values)
x = x.reshape(-1,x.shape[-1])

plt.figure()
plt.fill_between(bin_centers, np.percentile(x, 16, axis=0), np.percentile(x, 84, axis=0), color='r', alpha=0.3)
plt.plot(bin_centers, np.median(x, axis=0), color='r', lw=2, label='posterior')
plt.plot(bin_centers, stats.beta(0.9,3.0).pdf(bin_centers), 'k--')
plt.ylim(0, None)
plt.xlim(-0.1, 1)
plt.yticks([])
plt.xlabel("e", fontsize=24)
plt.ylabel("P(e)", fontsize=24)
plt.legend(fontsize=16)
plt.show()

In [ ]:
_ = corner.corner(trace, var_names=['log_s', 'log_r'])

## Re-weight posteriors

In [ ]:
keys = 'koi ror b T14'.split()
simulated = {}
posterior = {}

for k in keys:
    simulated[k] = np.zeros(len(results), dtype='float')
    posterior[k+'_16'] = np.zeros(len(results), dtype='float')
    posterior[k+'_50'] = np.zeros(len(results), dtype='float')
    posterior[k+'_84'] = np.zeros(len(results), dtype='float')

for i, d in enumerate(data):
    koi = 'K' + results[i].target[1:]
    use = truth.koi_id == koi
    
    w = d.WEIGHTS.values * stats.beta(1.3, 2.7).pdf(d.ECC.values)
    w /= np.sum(w)

    simulated['ror'][i] = truth[use].ror
    simulated['b'][i] = truth[use].impact
    simulated['T14'][i] = truth[use].duration
    
    posterior['ror_16'][i] = weighted_percentile(d.ROR.values, 16, w=w)
    posterior['ror_50'][i] = weighted_percentile(d.ROR.values, 50, w=w)
    posterior['ror_84'][i] = weighted_percentile(d.ROR.values, 84, w=w)
    
    posterior['b_16'][i] = weighted_percentile(d.IMPACT.values, 16, w=w)
    posterior['b_50'][i] = weighted_percentile(d.IMPACT.values, 50, w=w)
    posterior['b_84'][i] = weighted_percentile(d.IMPACT.values, 84, w=w)
    
    posterior['T14_16'][i] = weighted_percentile(d.DUR14.values, 16, w=w)
    posterior['T14_50'][i] = weighted_percentile(d.DUR14.values, 50, w=w)
    posterior['T14_84'][i] = weighted_percentile(d.DUR14.values, 84, w=w)

In [ ]:
for k in keys:
    if k!= 'koi':
        fig, ax = plt.subplots(1,2, figsize=(12,5))

        xmin = simulated[k].min()
        xmax = simulated[k].max()
        x = np.linspace(xmin, xmax)
        
        yerr1 = posterior[k+'_50'] - posterior[k+'_16']
        yerr2 = posterior[k+'_84'] - posterior[k+'_50']
        
        zscore = (posterior[k+'_50'] - simulated[k])/(0.5*(posterior[k+'_84'] - posterior[k+'_16']))
        
        ax[0].errorbar(simulated[k], posterior[k+'_50'], yerr=(yerr1, yerr2), fmt='k.', alpha=0.5)
        ax[0].plot(x, x, 'r:')
        ax[0].set_xlabel(k, fontsize=20)
        ax[0].set_xlim(xmin, xmax)
        ax[0].set_ylim(0.9*xmin, 1.1*xmax)
        ax[1].hist(zscore, bins=np.linspace(-5,5,21), color='lightgrey')
        plt.show()